# Programação Genética - Trabalho Final
## Leonardo Augusto Ferreira
### leauferreira@cpdee.ufmg.br

## Alunos:
### Gabriel Camatta Zanotelli - 2018020140
### Lucas de Almeida Martins - 2018020328

In [10]:
# Anaconda
# !conda install gplearn
# !conda install eckity
# !conda install pmlb

# python - jupyter nootebook - colab
# !pip install gplearn
# !pip install eckity
# !pip install pmlb


# documentation:
# https://docs.sympy.org/latest/install.html
# https://github.com/EC-KitY/EC-KitY
# https://epistasislab.github.io/pmlb/

In [40]:
from pmlb import fetch_data
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from gplearn.genetic import SymbolicRegressor
import numpy as np

In [12]:
from eckity.algorithms.simple_evolution import SimpleEvolution
from eckity.sklearn_compatible.sk_classifier import SKClassifier
from eckity.breeders.simple_breeder import SimpleBreeder
from eckity.creators.gp_creators.ramped_hh import RampedHalfAndHalfCreator
from eckity.genetic_encodings.gp.tree.functions import f_add, f_mul, f_sub, f_div, f_neg, f_sqrt, f_log, f_abs, f_inv, f_max, \
    f_min
from eckity.genetic_encodings.gp.tree.utils import create_terminal_set
from eckity.genetic_operators.crossovers.subtree_crossover import SubtreeCrossover
from eckity.genetic_operators.mutations.subtree_mutation import SubtreeMutation
from eckity.genetic_operators.selections.tournament_selection import TournamentSelection
from eckity.statistics.best_avg_worst_size_tree_statistics import BestAverageWorstSizeTreeStatistics
from eckity.subpopulation import Subpopulation
from eckity.termination_checkers.threshold_from_target_termination_checker import ThresholdFromTargetTerminationChecker

# Adding your own functions
from eckity.sklearn_compatible.classification_evaluator import ClassificationEvaluator


In [13]:
x, y = fetch_data('breast', return_X_y=True)

In [14]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=.7)

In [32]:
# terminal_set = create_terminal_set(x_train)
# # function_set = [f_add, f_mul, f_sub, f_div, f_sqrt, f_log, f_abs, f_neg, f_inv, f_max, f_min]
#
# function_set = [f_add, f_mul, f_sub, f_div]
#
# algo = SimpleEvolution(
#     Subpopulation(
#         creators=RampedHalfAndHalfCreator(init_depth=(2, 4),
#                                                     terminal_set=terminal_set,
#                                                     function_set=function_set,
#                                                     bloat_weight=0.0001),
#         population_size=1000,
#         evaluator=ClassificationEvaluator(),
#         higher_is_better=True,
#         elitism_rate=0.05,
#         operators_sequence=[
#             SubtreeCrossover(probability=0.9, arity=2),
#             SubtreeMutation(probability=0.2, arity=1)
#         ],
#         selection_methods=[
#             # (selection method, selection probability) tuple
#             (TournamentSelection(tournament_size=4, higher_is_better=True), 1)
#         ]
#     ),
#     breeder=SimpleBreeder(),
#     max_workers=1,
#     max_generation=100,
#
#     termination_checker=ThresholdFromTargetTerminationChecker(optimal=1, threshold=0.03),
#     statistics=BestAverageWorstSizeTreeStatistics()
# )
#
# classifier = SKClassifier(algo)
#
# classifier.fit(x_train, y_train)

In [33]:
# acc = accuracy_score(y_test, classifier.predict(x_test))
# print(acc)

## Comparação de Desempenho entre as Bibliotecas de Programação Genética EC-KitY e GPlearn utilizando Datasets de Regressão e Classificação

Para o trabalho final, você deverá utilizar as bibliotecas de programação Genética EC-KitY e GPlearn, juntamente com a biblioteca Penn Machine Learning Benchmarks, a fim de gerar um código Python que realizará testes em três conjuntos de dados para regressão e três conjuntos de dados para classificação. Seu objetivo é comparar as métricas obtidas e determinar qual biblioteca apresenta os melhores resultados.

Durante o estudo, você deverá variar as probabilidades de mutação e cruzamento, a fim de identificar a melhor configuração para cada uma das bibliotecas. Registre cuidadosamente os resultados obtidos em relação às métricas avaliadas, considerando medidas como acurácia, precisão, recall, F1-score, erro médio quadrático (RMSE) ou outras relevantes para o problema em questão.

Ao final do estudo, apresente uma análise comparativa dos resultados para cada biblioteca, destacando as probabilidades de mutação e cruzamento que produziram os melhores desempenhos. Utilize gráficos, tabelas ou outras visualizações que considerar pertinentes para ilustrar seus resultados e facilitar a compreensão.

---
# Relatório

Nesta atividade analisamos a eficiência das bibliotecas de Programação Genética *EC-KitY* e *GPlearn* de forma a fazer uma análise quantitativa da eficiência de ambas, por meio da utilização de diversos conjuntos de dados disponíveis na biblioteca *pmlb*.

Esta análise será feita utilizando ambas as bibliotecas para resolução de problemas binários e mono objetivo de quatro conjuntos de dados selecionados, variando dois parâmetros de imensa importância: a probabilidade de mutação e cruzamento. Para cada combinação, serão realizadas diversas iterações, de forma a obter um resultado médio.


### Conjuntos de dados

In [38]:
x1, y1 = fetch_data('1089_USCrime', return_X_y=True)
x2, y2 = fetch_data('cloud', return_X_y=True)
x3, y3 = fetch_data('201_pol', return_X_y=True)
x4, y4 = fetch_data('poker', return_X_y=True)

### Desenvolvimento

Para ambas as bibliotecas serão adotados alguns parâmetros comuns e imutáveis ao longo dos testes realizados:
- População máxima de 1000 indivíduos
- Mesmo conjunto de equações para a resolução do problema (adição, subtração, multiplicação e divisão)
- Valores base de probabilidade de mutação como 70% e probabilidade de cruzamento e 100%
- Serão testados valores da probabilidade de mutação de 30-90%, com um passo de 20%
- Serão testados valores da probabilidade de cruzamento de 40-100%, com um passo de 20%
- Máximo de 100 gerações
- Para cada caso, serão feitas um número de execuções de garantam uma certa segurança dos resultados. Para essa ativiade, serão feitas 10 iterações

### Set-up

In [49]:
function_set = [f_add, f_mul, f_sub, f_div]
mutation_vec  = [0.3, 0.5, 0.7, 0.9]
crossover_vec = [0.4, 0.6, 0.8, 1]
iterations = 10
f = lambda x_0, x_1 : x_0**2 - x_1**2 + x_1 - 1

### EC-KitY

In [25]:
def ec_kity(x, y, _mutation = 0.7, _crossover = 1, _higher_is_better = True):

    x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=.7)
    terminal_set = create_terminal_set(x_train)
    algo = SimpleEvolution(
        Subpopulation(
            creators=RampedHalfAndHalfCreator(init_depth=(2, 4),
                                                        terminal_set=terminal_set,
                                                        function_set=function_set,
                                                        bloat_weight=0.0001),
            population_size=1000,
            evaluator=ClassificationEvaluator(),
            higher_is_better=_higher_is_better,
            elitism_rate=0.05,
            operators_sequence=[
                SubtreeCrossover(probability=_mutation, arity=2),
                SubtreeMutation(probability=_crossover, arity=1)
            ],
            selection_methods=[
                # (selection method, selection probability) tuple
                (TournamentSelection(tournament_size=4, higher_is_better=True), 1)
            ]
        ),
        breeder=SimpleBreeder(),
        max_workers=1,
        max_generation=100,

        termination_checker=ThresholdFromTargetTerminationChecker(optimal=1, threshold=0.03),
        statistics=BestAverageWorstSizeTreeStatistics()
    )

    classifier = SKClassifier(algo)

    classifier.fit(x_train, y_train)
    return accuracy_score(y_test, classifier.predict(x_test))

### GPLearn

In [47]:
def gp_learn(x, y, _mutation = 0.7, _crossover = 1, _higher_is_better = True):

    X_mot = np.random.uniform(-1, 1, 100).reshape(50, 2)
    y_mot = f(X_mot[:, 0], X_mot[:, 1])

    gplearn_model = SymbolicRegressor(population_size=5000,
                                      generations=100,
                                      function_set=('add', 'sub', 'mul', 'div'),
                                      stopping_criteria=0.01,
                                      p_crossover=_crossover,
                                      p_subtree_mutation=0.1,
                                      p_hoist_mutation=0.05,
                                      p_point_mutation=_mutation,
                                      max_samples=0.9,
                                      verbose=1,
                                      parsimony_coefficient=0.01,
                                      random_state=0)
    gplearn_model.fit(X_mot, y_mot)

## Execução

In [52]:
# Dados
usc_ec_kity_cross_results = []; usc_ec_kity_mutt_results  = []
cloud_ec_kity_cross_results = []; cloud_ec_kity_mutt_results  = []
col_ec_kity_cross_results = []; col_ec_kity_mutt_results  = []
poker_ec_kity_cross_results = []; poker_ec_kity_mutt_results  = []

usc_gp_learn_cross_results = []; usc_gp_learn_mutt_results  = []
cloud_gp_learn_cross_results = []; cloud_gp_learn_mutt_results  = []
col_gp_learn_cross_results = []; col_gp_learn_mutt_results  = []
poker_gp_learn_cross_results = []; poker_gp_learn_mutt_results  = []

# US Crime
for cross in crossover_vec:
    usc_ec_kity_cross_results.append([ec_kity(x1, y1, _crossover=cross, _higher_is_better=False) for _ in range(iterations)])
    usc_gp_learn_cross_results.append([gp_learn(x1, y1, _crossover=cross, _higher_is_better=False) for _ in range(iterations)])
for mut in mutation_vec:
    usc_ec_kity_mutt_results.append( [ec_kity(x1, y1, _mutation=mut, _higher_is_better=False) for _ in range(iterations)])
    usc_gp_learn_mutt_results.append( [gp_learn(x1, y1, _mutation=mut, _higher_is_better=False) for _ in range(iterations)])


KeyboardInterrupt: 

### Anotações

Bases de dados de classificação tem que ser binaria
As de regressão tem que ser mono objetivo

Usar a biblioteca "pmlb" pra pegar um banco de dados

Variar as probabilidades ditas no enunciado

Relatorio no markdown

Rodar umas 30 (ou 3) vezes pra cada resultado (uma quantidade que de uma certa segurnaça) uma vez que o resultado é probabilistico

Pegar uns 3 ou 4 valores diferentes para cada